# JPEG Defense

In this exercise, you will construct a defense against one of the `cleverhans` attacks by using JPEG compression on the adversarial examples before classification.

## Notes
* Don't be surprised if the accuracy isn't much better after your JPEG defense (+6% is good enough). Why do you think the JPEG defense is less effective with MNIST?
* If you see a warning that there are no GPUs and CPUs are being used, ensure `nb_epochs=1`.
* I recommend attacking the FGSM code in the notebook.
* Useful TensorFlow functions are:
  * `tf.image.encode_jpeg`
  * `tf.image.decode_jpeg`
  * `tf.image.convert_image_dtype`
* Remember! MNIST images are in grayscale.
* If you see errors like `ValueError: ... that name is already used` you may need to call `tf.reset_default_graph()`.

In [ ]:
# DO NOT REMOVE. This allows MNIST dataset loading without Internet access.
import sys
sys.path.append('../util')
import dataset
sys.modules['cleverhans.dataset'] = dataset

# Add conversion code here!

In [11]:
def jpeg_compress(x, quality=75):
    """JPEG encode/decode `x` at the specified `quality`."""
    return x

In [12]:
# Copied from cleverhans/cleverhans_tutorials/mnist_tutorial_tf.py
"""
This tutorial shows how to generate adversarial examples using FGSM
and train a model using adversarial training with TensorFlow.
It is very similar to mnist_tutorial_keras_tf.py, which does the same
thing but with a dependence on keras.
The original paper can be found at:
https://arxiv.org/abs/1412.6572
"""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

import logging
import numpy as np
import tensorflow as tf

from cleverhans.loss import CrossEntropy
from cleverhans.dataset import MNIST
from cleverhans.utils_tf import model_eval
from cleverhans.train import train
from cleverhans.attacks import FastGradientMethod
from cleverhans.utils import AccuracyReport, set_log_level
from cleverhans_tutorials.tutorial_models import ModelBasicCNN

NB_EPOCHS = 6
BATCH_SIZE = 128
LEARNING_RATE = 0.001
CLEAN_TRAIN = True
BACKPROP_THROUGH_ATTACK = False
NB_FILTERS = 64


def mnist_tutorial(train_start=0, train_end=60000, test_start=0,
                   test_end=10000, nb_epochs=NB_EPOCHS, batch_size=BATCH_SIZE,
                   learning_rate=LEARNING_RATE,
                   clean_train=CLEAN_TRAIN,
                   testing=False,
                   backprop_through_attack=BACKPROP_THROUGH_ATTACK,
                   nb_filters=NB_FILTERS, num_threads=None,
                   label_smoothing=0.1):
  """
  MNIST cleverhans tutorial
  :param train_start: index of first training set example
  :param train_end: index of last training set example
  :param test_start: index of first test set example
  :param test_end: index of last test set example
  :param nb_epochs: number of epochs to train model
  :param batch_size: size of training batches
  :param learning_rate: learning rate for training
  :param clean_train: perform normal training on clean examples only
                      before performing adversarial training.
  :param testing: if true, complete an AccuracyReport for unit tests
                  to verify that performance is adequate
  :param backprop_through_attack: If True, backprop through adversarial
                                  example construction process during
                                  adversarial training.
  :param label_smoothing: float, amount of label smoothing for cross entropy
  :return: an AccuracyReport object
  """

  # Object used to keep track of (and return) key accuracies
  report = AccuracyReport()

  # Set TF random seed to improve reproducibility
  tf.set_random_seed(1234)

  # Set logging level to see debug information
  set_log_level(logging.DEBUG)

  # Create TF session
  if num_threads:
    config_args = dict(intra_op_parallelism_threads=1)
  else:
    config_args = {}
  sess = tf.Session(config=tf.ConfigProto(**config_args))

  # Get MNIST data
  mnist = MNIST(train_start=train_start, train_end=train_end,
                test_start=test_start, test_end=test_end)
  x_train, y_train = mnist.get_set('train')
  x_test, y_test = mnist.get_set('test')

  # Use Image Parameters
  img_rows, img_cols, nchannels = x_train.shape[1:4]
  nb_classes = y_train.shape[1]

  # Define input TF placeholder
  x = tf.placeholder(tf.float32, shape=(None, img_rows, img_cols,
                                        nchannels))
  y = tf.placeholder(tf.float32, shape=(None, nb_classes))

  # Train an MNIST model
  train_params = {
      'nb_epochs': nb_epochs,
      'batch_size': batch_size,
      'learning_rate': learning_rate
  }
  eval_params = {'batch_size': batch_size}
  fgsm_params = {
      'eps': 0.3,
      'clip_min': 0.,
      'clip_max': 1.
  }
  rng = np.random.RandomState([2017, 8, 30])

  def do_eval(preds, x_set, y_set, report_key, report_text=None):
    acc = model_eval(sess, x, y, preds, x_set, y_set, args=eval_params)
    setattr(report, report_key, acc)
    if report_text:
      print('Test accuracy on %s examples: %0.4f' % (report_text, acc))

  if clean_train:
    model = ModelBasicCNN('model1', nb_classes, nb_filters)
    preds = model.get_logits(x)
    loss = CrossEntropy(model, smoothing=label_smoothing)

    def evaluate():
      do_eval(preds, x_test, y_test, 'clean_train_clean_eval', 'legitimate')

    train(sess, loss, x_train, y_train, evaluate=evaluate,
          args=train_params, rng=rng, var_list=model.get_params())

    # Calculate training error
    if testing:
      do_eval(preds, x_train, y_train, 'train_clean_train_clean_eval', None)

    # Initialize the Fast Gradient Sign Method (FGSM) attack object and
    # graph
    fgsm = FastGradientMethod(model, sess=sess)
    adv_x = fgsm.generate(x, **fgsm_params)
    preds_adv = model.get_logits(adv_x)

    # Evaluate the accuracy of the MNIST model on adversarial examples
    do_eval(preds_adv, x_test, y_test, 'clean_train_adv_eval', 'adversarial')

    # Calculate training error
    if testing:
      do_eval(preds_adv, x_train, y_train, 'train_clean_train_adv_eval', None)
    
    # JPEG evaluation happens here!
    for quality in range(10, 100, 5):
        from functools import partial
        fn = partial(jpeg_compress, quality=quality)
        jpeg_adv_x = tf.map_fn(fn, tf.image.convert_image_dtype(adv_x, dtype=tf.uint8))
        converted = tf.image.convert_image_dtype(jpeg_adv_x, dtype=tf.float32)
        converted.set_shape((None, 28, 28, 1))
        preds_jpeg_adv = model.get_logits(converted)
        do_eval(preds_jpeg_adv, x_test, y_test, 'clean_train_adv_eval', 'jpeg (quality = %d)' % quality)

# Last loop above is where your `jpeg_compress` function is called

In [13]:
def main(argv=None):
  from cleverhans_tutorials import check_installation
  check_installation(__file__)

  mnist_tutorial(nb_epochs=NB_EPOCHS, batch_size=BATCH_SIZE,
                 learning_rate=LEARNING_RATE,
                 clean_train=CLEAN_TRAIN,
                 backprop_through_attack=BACKPROP_THROUGH_ATTACK,
                 nb_filters=NB_FILTERS)

In [ ]:
tf.reset_default_graph()
mnist_tutorial(nb_epochs=1)